참고 페이지:
* https://deep-learning-study.tistory.com/398?category=963091

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 실패작
* init에 컴포넌트 정의시, 한번선언된 layer는 객체로 인식이되어 그걸 forward에서 중복으로 사용하면 안됌.
* 중복으로 사용시, 한 layer로 인식됌

In [2]:
class VGG19(nn.Module):
    def __init__(self, in_channels=3, num_classes=10):
        super().__init__()
        self.in_channels = in_channels
        self.num_classes = num_classes
        
        self.conv1 = nn.Conv2d(self.in_channels, 64, 3, 1, 1)
        self.conv2 = nn.Conv2d(64, 64, 3, 1, 1)
        self.b1 = nn.BatchNorm2d(64)
        
        self.conv3 = nn.Conv2d(64, 128, 3, 1, 1)
        self.conv4 = nn.Conv2d(128, 128, 3, 1, 1)
        self.b2 = nn.BatchNorm2d(128)
        
        self.conv5 = nn.Conv2d(128, 256, 3, 1, 1)
        self.conv6_8 = nn.Conv2d(256, 256, 3, 1, 1)
        self.b3 = nn.BatchNorm2d(256)
        
        self.conv9 = nn.Conv2d(256, 512, 3, 1, 1)
        self.conv10_16 = nn.Conv2d(512, 512, 3, 1, 1)
        self.b4 = nn.BatchNorm2d(512)
        
        self.fc1 = nn.Linear(512*7*7, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, self.num_classes)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.act = nn.ReLU()
        self.do = nn.Dropout()
        
        
    def forward(self, x):
        x = self.act(self.b1(self.conv1(x)))
        x = self.act(self.b1(self.conv2(x)))
        x = self.pool(x)
        
        x = self.act(self.b2(self.conv3(x)))
        x = self.act(self.b2(self.conv4(x)))
        x = self.pool(x)
        
        x = self.act(self.b3(self.conv5(x)))
        x = self.act(self.b3(self.conv6_8(x)))
        x = self.act(self.b3(self.conv6_8(x)))
        x = self.act(self.b3(self.conv6_8(x)))
        x = self.pool(x)
        
        x = self.act(self.b4(self.conv9(x)))
        x = self.act(self.b4(self.conv10_16(x)))
        x = self.act(self.b4(self.conv10_16(x)))
        x = self.act(self.b4(self.conv10_16(x)))
        x = self.pool(x)
        
        x = self.act(self.b4(self.conv10_16(x)))
        x = self.act(self.b4(self.conv10_16(x)))
        x = self.act(self.b4(self.conv10_16(x)))
        x = self.act(self.b4(self.conv10_16(x)))
        x = self.pool(x)
        
        x = self.do(self.act(self.fc1(x)))
        x = self.do(self.act(self.fc2(x)))
        x = self.do(self.act(self.fc3(x)))
        
        return x
        

In [5]:
model = VGG19(in_channels=3, num_classes=10)
print(model)

VGG19(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (b1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (b2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6_8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (b3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv9): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv10_16): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (b4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1

In [9]:
VGG_TYPES = {
    "vgg19": [64, 64, "MP", 128, 128, "MP", 256, 256, 256, 256, "MP", 512, 512, 512, 512, "MP", 512, 512, 512, 512, "MP"]
}
class VGGNET(nn.Module):
    def __init__(self, model_type, in_channels=3, num_classes=10):
        super().__init__()
        self.in_channels = in_channels
        self.num_classes = num_classes
        self.feature_extractor_layers = self.create_feature_extractor(VGG_TYPES[model_type])
        self.fc_layers = self.create_fully_conneted_layers()
    
    def create_fully_conneted_layers(self):
        return nn.Sequential(
            nn.Linear(512*1*1, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, self.num_classes),
        )
        
    def create_feature_extractor(self, layer_list):
        layers = []
        in_channels = self.in_channels
        for each in layer_list:
            out_channels = each
            if type(each) == int:
                layers += [
                    nn.Conv2d(in_channels, out_channels, 3, 1, 1),
                    nn.BatchNorm2d(each),
                    nn.ReLU()
                ]
                in_channels = out_channels
            elif each == "MP":
                layers += [nn.MaxPool2d(2, 2)]
                
        return nn.Sequential(*layers)
        
    def forward(self, x):
        x = self.feature_extractor_layers(x)
        x = self.fc_layers(x)
        
        return x
        

In [11]:
model = VGGNET(model_type="vgg19", in_channels=3, num_classes=10)
print(model)

VGGNET(
  (feature_extractor_layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3), stride=

In [6]:
# VGG type dict
# int : output chnnels after conv layer
# 'M' : max pooling layer
VGG_types = {
    'VGG11' : [64, 'M', 128, 'M', 256, 256, 'M', 512,512, 'M',512,512,'M'],
    'VGG13' : [64,64, 'M', 128, 128, 'M', 256, 256, 'M', 512,512, 'M', 512,512,'M'],
    'VGG16' : [64,64, 'M', 128, 128, 'M', 256, 256,256, 'M', 512,512,512, 'M',512,512,512,'M'],
    'VGG19' : [64,64, 'M', 128, 128, 'M', 256, 256,256,256, 'M', 512,512,512,512, 'M',512,512,512,512,'M']
}

class VGG(nn.Module):
    def __init__(self, in_channels=3, num_classes=10, model_type='VGG19'):
        super().__init__()
        self.in_channels = in_channels
        self.num_classes = num_classes
        self.conv_layers = self.create_conv_layers(VGG_types[model_type])
        self.fcs = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, num_classes)
        )
        
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(-1, 512*7*7)
        x = self.fcs(x)
        return x
    
    def create_conv_layers(self, architecture):
        layers = []
        in_channels = self.in_channels
        
        for x in architecture:
            if type(x) == int:  # int means convlayer
                out_channels = x
                
                layers += [
                    nn.Conv2d(
                        in_channels=in_channels, out_channels=out_channels, 
                        kernel_size=(3,3), stride=(1,1), padding=(1,1)),
                    nn.BatchNorm2d(x),
                    nn.ReLU()
                ]
                
                in_channels = x
            elif x == 'M':
                layers += [nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))]
        
        return nn.Sequential(*layers)

In [4]:
model = VGG(model_type="VGG19", in_channels=3, num_classes=10)
print(model)

VGG(
  (conv_layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3), str